In [57]:
import numpy as np
import pandas as pd
import torch


from JSE.data import *
from JSE.settings import data_info, optimizer_info
from JSE.models import *
from JSE.training import *

import argparse
import os
import sys

In [40]:
def run_JSE_for_concept(data_obj,device, dataset, seed, batch_size, optimizer_settings, lr, weight_decay, epochs, alpha, expected_diff, early_stopping, per_step=50, null_is_concept=False, eval_balanced=True, solver='SGD', balanced_weights_concept=True, which_concept='First'):
    # Load the data
    X_train, X_val, X_test = data_obj.X_train, data_obj.X_val, data_obj.X_test
    y_m_test = data_obj.y_m_test
    y_c_test = data_obj.y_c_test
    y_c_2_train = data_obj.y_c_2_train
    y_c_2_val = data_obj.y_c_2_val
    y_c_2_test = data_obj.y_c_2_test


    # set y_c_train, y_c_val, y_c_test to y_c_2_train, y_c_2_val, y_c_2_test
    if which_concept == 'Second':
        data_obj.y_c_train = y_c_2_train.clone()
        data_obj.y_c_val = y_c_2_val.clone()
        data_obj.y_c_test = y_c_2_test.clone()

    # define the weights for the second concept
    if balanced_weights_concept:
        if which_concept == 'First':
            concept_weights_train, concept_weights_val = data_obj.get_class_weights_train_val(y_c_test, y_c_test)
        else:
            concept_weights_train, concept_weights_val = data_obj.get_class_weights_train_val(y_c_2_train, y_c_2_val)
        
        include_weights = True
    else:
        concept_weights_train, concept_weights_val = None, None
        include_weights = False

    # define the loaders
    concept_first = True
    loaders = data_obj.create_loaders(batch_size=batch_size, workers=0, with_concept=True, include_weights=include_weights, train_weights=concept_weights_train, val_weights=concept_weights_val,concept_first=concept_first )

    
    # Run the JSE algorithm
    set_seed(seed)
    V_c_1, V_m_1, d_c_1, d_m_1 = train_JSE(data_obj,
                                                            device=device,
                                                            batch_size=batch_size, 
                                                            solver=solver,
                                                            lr=lr,
                                                            per_step=per_step,
                                                            tol=optimizer_settings['tol'],
                                                            early_stopping=early_stopping,
                                                            patience=optimizer_settings['patience'],
                                                            epochs=epochs, 
                                                            Delta = expected_diff,
                                                            alpha=alpha,
                                                            null_is_concept = null_is_concept,
                                                            eval_balanced=eval_balanced, 
                                                            weight_decay=weight_decay,
                                                            include_weights=include_weights,
                                                            train_weights=concept_weights_train,
                                                            val_weights=concept_weights_val,
                                                            model_base_name='JSE_first_part_'+str(seed)+'_',
                                                            concept_first=concept_first,
                                                            )
    
    # use the V_c from the outer loop
    d = V_c_1.shape[0]
    P_c_1_orth = torch.eye(d) - create_P(V_c_1) 

    # transform the data for the first concept
    X_train_transformed_1 = torch.matmul(X_train, P_c_1_orth)
    X_val_transformed_1 = torch.matmul(X_val, P_c_1_orth)
    X_test_transformed_1 = torch.matmul(X_test, P_c_1_orth)
    data_obj.reset_X(X_train_transformed_1, X_val_transformed_1, batch_size=batch_size, reset_X_objects=True, include_weights=False, train_weights = None, val_weights = None, only_main=True)

    # Train the model to get main
    settings = 'standard_ERM_settings'
    lr_ERM = optimizer_info[settings][dataset]['lr']
    weight_decay_ERM = optimizer_info[settings][dataset]['weight_decay']

     # Train the model to get main
    set_seed(seed)
    main_model = return_linear_model(d, 
                                                data_obj.main_loader,
                                                device,
                                                solver = solver,
                                                lr=lr_ERM,
                                                per_step=per_step,
                                                tol = optimizer_settings['tol'],
                                                early_stopping = early_stopping,
                                                patience = optimizer_settings['patience'],
                                                epochs = epochs,
                                                bias=True,
                                                weight_decay=weight_decay_ERM, 
                                                model_name='main_model_after_JSE_second_concept'+str(seed),
                                                save_best_model=True
                                                )
        

    # get the accuracy of the main model
    y_m_pred_test = main_model(X_test_transformed_1)

    # get the accuracy of the main model overall 
    main_acc_after = get_acc_pytorch_model(y_m_test, y_m_pred_test)

    # get the accuracy of the main model per group - first concept
    result_per_group_1, _ = get_acc_per_group(y_m_pred_test, y_m_test, y_c_test)

    # get the accuracy of the main model per group - second concept
    result_per_group_2, _ = get_acc_per_group(y_m_pred_test, y_m_test, y_c_2_test)

    # save the results dict
    results_dict = {
        'main_acc_after': main_acc_after,
        'result_per_group_1': result_per_group_1,
        'result_per_group_2': result_per_group_2,
        'V_c': V_c_1,
    }
    
    return results_dict




In [ ]:
dataset = 'celebA'
dataset_setting = 'sampled_data_two_concepts'
device = 'cpu'
spurious_ratio=0.8
single_finetuned_model = False
demean=True
pca=True 
k_components=300
batch_size = 128
lr = 0.001
weight_decay = 0.001
epochs = 50
alpha = 0.05
expected_diff = 0.0
early_stopping = True

 # set the settings for dataset
dataset_settings = data_info[dataset][dataset_setting]
optimizer_settings = optimizer_info['All']


list_results_JSE_first_concept = []
for seed in range(0, 5):
    # get the data obj for the first concept
    set_seed(seed)
    data_obj_first_concept = get_dataset_obj(dataset, dataset_settings, spurious_ratio, data_info, seed, device,  single_model_for_embeddings=single_finetuned_model)

    # demean, pca for the first concept
    if demean:
        data_obj_first_concept.demean_X(reset_mean=True, include_test=True)
    if pca:
        data_obj_first_concept.transform_data_to_k_components(k_components, reset_V_k=True, include_test=True)
        V_k_train = data_obj_first_concept.V_k_train

    # run JSE for the first concept
    result_dict_JSE_first_concept = run_JSE_for_concept(data_obj_first_concept,device, 'celebA', seed, batch_size, optimizer_settings, lr, weight_decay, epochs, alpha, expected_diff, early_stopping, which_concept='First', balanced_weights_concept=False)
    list_results_JSE_first_concept.append(result_dict_JSE_first_concept)



In [ ]:
dataset = 'celebA'
dataset_setting = 'sampled_data_two_concepts'
device = 'cpu'
spurious_ratio=0.8
single_finetuned_model = False
demean=True
pca=True 
k_components=300
batch_size = 128
lr = 0.01
weight_decay = 0.01
epochs = 50
alpha = 0.05
expected_diff = 0.0
early_stopping = True

 # set the settings for dataset
dataset_settings = data_info[dataset][dataset_setting]
optimizer_settings = optimizer_info['All']


list_results_JSE_second_concept = []
for seed in range(0, 5):
    # get the data obj for the second concept
    set_seed(seed)
    data_obj_second_concept = get_dataset_obj(dataset, dataset_settings, spurious_ratio, data_info, seed, device,  single_model_for_embeddings=single_finetuned_model)

    # demean, pca for the second concept
    if demean:
        data_obj_second_concept.demean_X(reset_mean=True, include_test=True)
    if pca:
        data_obj_second_concept.transform_data_to_k_components(k_components, reset_V_k=True, include_test=True)
        V_k_train = data_obj_second_concept.V_k_train

    # run JSE for the second concept
    result_dict_JSE_second_concept = run_JSE_for_concept(data_obj_second_concept,device, 'celebA', seed, batch_size, optimizer_settings, lr, weight_decay, epochs, alpha, expected_diff, early_stopping, which_concept='Second', balanced_weights_concept=True)
    list_results_JSE_second_concept.append(result_dict_JSE_second_concept)



In [ ]:
dataset = 'celebA'
dataset_setting = 'sampled_data_two_concepts'
device = 'cpu'
spurious_ratio=0.8
single_finetuned_model = False
demean=True
pca=True 
k_components=300
batch_size = 128
settings = 'standard_ERM_settings'
lr_ERM = optimizer_info[settings][dataset]['lr']
weight_decay_ERM = optimizer_info[settings][dataset]['weight_decay']
epochs = 50
alpha = 0.05
expected_diff = 0.0
early_stopping = True
solver='SGD'
per_step=50


 # set the settings for dataset
dataset_settings = data_info[dataset][dataset_setting]
optimizer_settings = optimizer_info['All']


# get the data obj
list_results_JSE_both = []
for seed in range(0, 5):

    # get the data obj
    set_seed(seed)
    data_obj = get_dataset_obj(dataset, dataset_settings, spurious_ratio, data_info, seed, device,  single_model_for_embeddings=single_finetuned_model)

    # demean, pca
    if demean:
        data_obj.demean_X(reset_mean=True, include_test=True)
    if pca:
        data_obj.transform_data_to_k_components(k_components, reset_V_k=True, include_test=True)
        V_k_train = data_obj.V_k_train

    # get the X_train, X_val, X_test
    X_train, X_val, X_test = data_obj.X_train, data_obj.X_val, data_obj.X_test
    y_m_test = data_obj.y_m_test
    y_c_test = data_obj.y_c_test
    y_c_2_test = data_obj.y_c_2_test

    # get the V_c_1, V_c_2
    V_c_1 = list_results_JSE_first_concept[seed]['V_c']
    V_c_2 = list_results_JSE_second_concept[seed]['V_c']
    P_1 = torch.eye(V_c_1.shape[0]) - create_P(V_c_1)
    P_2 = torch.eye(V_c_2.shape[0]) - create_P(V_c_2)
    d = V_c_1.shape[0]

    # transform the data to remove first concept, then the second concept
    X_train_transformed = torch.matmul(torch.matmul(X_train, P_1), P_2)
    X_val_transformed = torch.matmul(torch.matmul(X_val, P_1), P_2)
    X_test_transformed = torch.matmul(torch.matmul(X_test, P_1), P_2)
    data_obj.reset_X(X_train_transformed, X_val_transformed, batch_size=batch_size, reset_X_objects=True, include_weights=False, train_weights = None, val_weights = None, only_main=True)



    # Train the model to get main
    set_seed(seed)
    main_model_JSE_both = return_linear_model(d, 
                                               data_obj.main_loader,
                                              device,
                                              solver = solver,
                                              lr=lr_ERM,
                                              per_step=per_step,
                                              tol = optimizer_settings['tol'],
                                              early_stopping = early_stopping,
                                              patience = optimizer_settings['patience'],
                                              epochs = epochs,
                                              bias=True,
                                              weight_decay=weight_decay_ERM, 
                                              model_name=dataset+'_main_model')
    
    # get the accuracy of the main model
    y_m_pred_test_ERM = main_model_JSE_both(X_test)

    # get the accuracy of the main model overall 
    main_acc_after = get_acc_pytorch_model(y_m_test, y_m_pred_test_ERM)

    # get the accuracy of the main model per group - first concept
    result_per_group_1, _ = get_acc_per_group(y_m_pred_test_ERM, y_m_test, y_c_test)

    # get the accuracy of the main model per group - second concept
    result_per_group_2, _ = get_acc_per_group(y_m_pred_test_ERM, y_m_test, y_c_2_test)

    result_dict_JSE_both = {
        'main_acc_after': main_acc_after,
        'result_per_group_1': result_per_group_1,
        'result_per_group_2': result_per_group_2}
    
    list_results_JSE_both.append(result_dict_JSE_both)
    

    

    


In [ ]:
dataset = 'celebA'
dataset_setting = 'sampled_data_two_concepts'
device = 'cpu'
spurious_ratio=0.8
single_finetuned_model = False
demean=True
pca=True 
k_components=300
balanced_training_second_concept = False
batch_size = 128
settings = 'standard_ERM_settings_GW'
lr_ERM = optimizer_info[settings][dataset]['lr']
weight_decay_ERM = optimizer_info[settings][dataset]['weight_decay']
epochs = 50
expected_diff = 0.0
early_stopping = True
solver='SGD'
per_step=50


 # set the settings for dataset
dataset_settings = data_info[dataset][dataset_setting]
optimizer_settings = optimizer_info['All']


# get the data obj
list_results_GW_ERM = []
for seed in range(0, 5):

    # get the data obj
    set_seed(seed)
    data_obj = get_dataset_obj(dataset, dataset_settings, spurious_ratio, data_info, seed, device,  single_model_for_embeddings=single_finetuned_model)

    # demean, pca
    if demean:
        data_obj.demean_X(reset_mean=True, include_test=True)
    if pca:
        data_obj.transform_data_to_k_components(k_components, reset_V_k=True, include_test=True)
        V_k_train = data_obj.V_k_train

    # get the X_train, X_val, X_test
    X_train, X_val, X_test = data_obj.X_train, data_obj.X_val, data_obj.X_test

    # get the y_m_test
    y_m_test = data_obj.y_m_test
    y_c_test = data_obj.y_c_test
    y_c_2_test = data_obj.y_c_2_test

    # use group-balance weights for the second concept
    include_weights = True
    main_weights_train, main_weights_val = data_obj.get_group_weights(normalized=True, second_concept=True)
    
    # reset the data objects
    data_obj.reset_X(X_train, X_val, batch_size=batch_size, reset_X_objects=False, include_weights=include_weights, train_weights = main_weights_train, val_weights = main_weights_val)
    d = X_train.shape[1]

    # Train the model to get main
    set_seed(seed)
    main_model_GW_ERM = return_linear_model(d, 
                                               data_obj.main_loader,
                                              device,
                                              solver = solver,
                                              lr=lr_ERM,
                                              per_step=per_step,
                                              tol = optimizer_settings['tol'],
                                              early_stopping = early_stopping,
                                              patience = optimizer_settings['patience'],
                                              epochs = epochs,
                                              bias=True,
                                              weight_decay=weight_decay_ERM, 
                                              model_name=dataset+'_main_model')
    
    # get the accuracy of the main model
    y_m_pred_test_GW = main_model_GW_ERM(X_test)

    # get the accuracy of the main model overall 
    main_acc_after = get_acc_pytorch_model(y_m_test, y_m_pred_test_GW)

    # get the accuracy of the main model per group - first concept
    result_per_group_1, _ = get_acc_per_group(y_m_pred_test_GW, y_m_test, y_c_test)

    # get the accuracy of the main model per group - second concept
    result_per_group_2, _ = get_acc_per_group(y_m_pred_test_GW, y_m_test, y_c_2_test)

    result_dict_GW_ERM = {
        'main_acc_after': main_acc_after,
        'result_per_group_1': result_per_group_1,
        'result_per_group_2': result_per_group_2}
    
    list_results_GW_ERM.append(result_dict_GW_ERM)
    

    

    


In [ ]:
dataset = 'celebA'
dataset_setting = 'sampled_data_two_concepts'
device = 'cpu'
spurious_ratio=0.8
single_finetuned_model = False
demean=True
pca=True 
k_components=300
balanced_training_second_concept = False
batch_size = 128
settings = 'standard_ERM_settings'
lr_ERM = optimizer_info[settings][dataset]['lr']
weight_decay_ERM = optimizer_info[settings][dataset]['weight_decay']
epochs = 50
alpha = 0.05
expected_diff = 0.0
early_stopping = True
solver='SGD'
per_step=50


 # set the settings for dataset
dataset_settings = data_info[dataset][dataset_setting]
optimizer_settings = optimizer_info['All']


# get the data obj

list_results_ERM = []
for seed in range(0, 5):

    # get the data obj
    set_seed(seed)
    data_obj = get_dataset_obj(dataset, dataset_settings, spurious_ratio, data_info, seed, device,  single_model_for_embeddings=single_finetuned_model)

    # demean, pca
    if demean:
        data_obj.demean_X(reset_mean=True, include_test=True)
    if pca:
        data_obj.transform_data_to_k_components(k_components, reset_V_k=True, include_test=True)
        V_k_train = data_obj.V_k_train

    # get the X_train, X_val, X_test
    X_train, X_val, X_test = data_obj.X_train, data_obj.X_val, data_obj.X_test

    # get the y_m_test
    y_m_test = data_obj.y_m_test
    y_c_test = data_obj.y_c_test
    y_c_2_test = data_obj.y_c_2_test

    # use group-balance weights for the second concept
    include_weights = False
    main_weights_train, main_weights_val = None, None

    # reset the data objects
    data_obj.reset_X(X_train, X_val, batch_size=batch_size, reset_X_objects=False, include_weights=include_weights, train_weights = main_weights_train, val_weights = main_weights_val)
    d = X_train.shape[1]

    # Train the model to get main
    set_seed(seed)
    main_model_ERM = return_linear_model(d, 
                                               data_obj.main_loader,
                                              device,
                                              solver = solver,
                                              lr=lr_ERM,
                                              per_step=per_step,
                                              tol = optimizer_settings['tol'],
                                              early_stopping = early_stopping,
                                              patience = optimizer_settings['patience'],
                                              epochs = epochs,
                                              bias=True,
                                              weight_decay=weight_decay_ERM, 
                                              model_name=dataset+'_main_model')
    
    # get the accuracy of the main model
    y_m_pred_test_ERM = main_model_ERM(X_test)

    # get the accuracy of the main model overall 
    main_acc_after = get_acc_pytorch_model(y_m_test, y_m_pred_test_ERM)

    # get the accuracy of the main model per group - first concept
    result_per_group_1, _ = get_acc_per_group(y_m_pred_test_ERM, y_m_test, y_c_test)

    # get the accuracy of the main model per group - second concept
    result_per_group_2, _ = get_acc_per_group(y_m_pred_test_ERM, y_m_test, y_c_2_test)

    result_dict_ERM = {
        'main_acc_after': main_acc_after,
        'result_per_group_1': result_per_group_1,
        'result_per_group_2': result_per_group_2}
    
    list_results_ERM.append(result_dict_ERM)
    

    

    


In [52]:
def create_table_entry_method(list_results, multiplier=100):

    # loop over the results
    acc = []
    worst_acc_1 =[]
    worst_acc_2 = []
    result_per_group_1 = []
    result_per_group_2 = []
    for result_dict in list_results:
        acc.append(result_dict['main_acc_after'])
        
        # from the results per group_1, get the worst accuracy
        wg_1 = result_dict['result_per_group_1']['mean'].values.min()
        wg_2 = result_dict['result_per_group_2']['mean'].values.min()
        worst_acc_1.append(wg_1)
        worst_acc_2.append(wg_2)
        
        # get the accuracy per group
        result_per_group_1.append(result_dict['result_per_group_1']['mean'].values)
        result_per_group_2.append(result_dict['result_per_group_2']['mean'].values)
    
    # get the average acc, and the se
    avg_acc = np.mean(acc)
    se_acc = np.std(acc)/np.sqrt(len(result_per_group_1))

    # get the average wg acc, and the se
    avg_worst_acc_1 = np.mean(worst_acc_1)
    se_worst_acc_1 = np.std(worst_acc_1)/np.sqrt(len(result_per_group_1))
    avg_worst_acc_2 = np.mean(worst_acc_2)
    se_worst_acc_2 = np.std(worst_acc_2)/np.sqrt(len(result_per_group_2))


    # get the average acc per group
    result_per_group_1 = np.array(result_per_group_1)
    result_per_group_2 = np.array(result_per_group_2)
    avg_acc_per_group_1 = np.mean(result_per_group_1, axis=0)
    avg_acc_per_group_2 = np.mean(result_per_group_2, axis=0)
    se_acc_per_group_1 = np.std(result_per_group_1, axis=0)/np.sqrt(len(result_per_group_1))
    se_acc_per_group_2 = np.std(result_per_group_2, axis=0)/np.sqrt(len(result_per_group_2))

    # create table entry - take the average accuracy and put the se in brackets
    acc = str(round(avg_acc*multiplier, 2)) + ' (' + str(round(se_acc*multiplier, 2)) + ')'
    worst_acc_1 = str(round(avg_worst_acc_1*multiplier, 2)) + ' (' + str(round(se_worst_acc_1*multiplier, 2)) + ')'
    worst_acc_2 = str(round(avg_worst_acc_2*multiplier, 2)) + ' (' + str(round(se_worst_acc_2*multiplier, 2)) + ')' 
    acc_per_group_1 = [str(round(avg_acc_per_group_1[i]*multiplier, 2)) + ' (' + str(round(se_acc_per_group_1[i]*multiplier, 2)) + ')' for i in range(len(avg_acc_per_group_1))]
    acc_per_group_2 = [str(round(avg_acc_per_group_2[i]*multiplier, 2)) + ' (' + str(round(se_acc_per_group_2[i]*multiplier, 2)) + ')' for i in range(len(avg_acc_per_group_2))]

    return acc, worst_acc_1, worst_acc_2, acc_per_group_1, acc_per_group_2,


def create_table(list_results_JSE_both, list_results_JSE_both_order_1, list_results_JSE_both_order_2, list_results_JSE_first_concept, list_results_JSE_second_concept, list_ERM_results, list_ERM_GW_results):

    # get the number of samples in test set
    n_samples_1 = list_results_JSE_both_order_1[0]['result_per_group_1']['count']
    n_samples_2 = list_results_JSE_both_order_1[0]['result_per_group_2']['count']

    # JSE with both concepts
    acc_JSE_both, worst_acc_1_JSE_both, worst_acc_2_JSE_both, acc_per_group_1_JSE_both, acc_per_group_2_JSE_both = create_table_entry_method(list_results_JSE_both)

    # JSE with first concept
    acc_JSE_first_concept, worst_acc_1_JSE_first_concept, worst_acc_2_JSE_first_concept, acc_per_group_1_JSE_first_concept, acc_per_group_2_JSE_first_concept = create_table_entry_method(list_results_JSE_first_concept)

    # JSE with second concept
    acc_JSE_second_concept, worst_acc_1_JSE_second_concept, worst_acc_2_JSE_second_concept, acc_per_group_1_JSE_second_concept, acc_per_group_2_JSE_second_concept = create_table_entry_method(list_results_JSE_second_concept)

    # GW-ERM
    acc_GW_ERM, worst_acc_1_GW_ERM, worst_acc_2_GW_ERM, acc_per_group_1_GW_ERM, acc_per_group_2_GW_ERM = create_table_entry_method(list_ERM_GW_results)

    # ERM
    acc_ERM, worst_acc_1_ERM, worst_acc_2_ERM, acc_per_group_1_ERM, acc_per_group_2_ERM = create_table_entry_method(list_ERM_results)

    # first table - all results for concept 1
    table = pd.DataFrame({
        'Method': [r'JSE (both)' , r'JSE ( $\ysp^{(1)}$)', r'JSE ( $\ysp^{(2)}$)', 'ERM', 'GW-ERM'],
        'Accuracy': [acc_JSE_both,  acc_JSE_first_concept, acc_JSE_second_concept, acc_ERM, acc_GW_ERM],
        'Worst group 1': [worst_acc_1_JSE_both, worst_acc_1_JSE_first_concept, worst_acc_1_JSE_second_concept, worst_acc_1_ERM, worst_acc_1_GW_ERM],
        'Worst group 2': [worst_acc_2_JSE_both, worst_acc_2_JSE_first_concept, worst_acc_2_JSE_second_concept, worst_acc_2_ERM, worst_acc_2_GW_ERM]})
    
    # second table - rows are groups, columns are methods, showing result per group
    table_per_group_1 = pd.DataFrame({
        'Group': ['Group 1', 'Group 2', 'Group 3', 'Group 4'],
        'n': n_samples_1,
        r'JSE (both)': acc_per_group_1_JSE_both,
        r'JSE ( $\ysp^{(1)}$)': acc_per_group_1_JSE_first_concept,
        r'JSE ( $\ysp^{(2)}$)': acc_per_group_1_JSE_second_concept,
        'ERM': acc_per_group_1_ERM,
        'GW-ERM': acc_per_group_1_GW_ERM})
    
    # third table - rows are groups, columns are methods, showing result per group
    table_per_group_2 = pd.DataFrame({
        'Group': ['Group 1', 'Group 2', 'Group 3', 'Group 4'],
        'n': n_samples_2,
        r'JSE (both)': acc_per_group_2_JSE_both,
        r'JSE ( $\ysp^{(1)}$)': acc_per_group_2_JSE_first_concept,
        r'JSE ( $\ysp^{(2)}$)': acc_per_group_2_JSE_second_concept,
        'ERM': acc_per_group_2_ERM,
        'GW-ERM': acc_per_group_2_GW_ERM})
    
    

    
    return table, table_per_group_1, table_per_group_2


table, table_per_group_1, table_per_group_2 = create_table(list_results_JSE_both, list_results_JSE_first_concept, list_results_JSE_second_concept, list_results_ERM, list_results_GW_ERM)



In [53]:

# order main-concept to be (1-1), (1-0), (0-1), (0-0)
table_per_group_1_ordered = table_per_group_1.copy()
table_per_group_2_ordered = table_per_group_2.copy()
table_per_group_1_ordered = table_per_group_1_ordered.iloc[[3, 2, 1, 0], :]
table_per_group_2_ordered = table_per_group_2_ordered.iloc[[3, 2, 1, 0], :]
